In [ ]:
# matrix coefficient creator for an i grid point
def matrix_cr(n,A):# A is the arthimetic of the constants
  tri_co,c,mat=-1-(2*A),n-2,[]
  d=c-3# number of zeros after real coefficients in matrix from second row
  one=[tri_co,A]+[0 for i in range(c-2)]
  last=[0 for i in range(c-2)]+[A,tri_co]
  for i in range(c-2):
    a=[0]*(i)+[A,tri_co,A]+[0]*(d-i)
    mat.append(a)
  return [one]+mat+[last]
matrix_cr(7,200)

[[401, -200, 0, 0, 0],
 [-200, 401, -200, 0, 0],
 [0, -200, 401, -200, 0],
 [0, 0, -200, 401, -200],
 [0, 0, 0, -200, 401]]

In [ ]:
def known_mat(arr,A):
  B=[]
  for i in range(len(arr)-2):
    a=arr[i+2]+arr[i]-2*arr[i+1]
    a=arr[i+1]+A*a
    B.append(-a)
  one,last=B[0]-A*arr[0], B[-1]-A*arr[-1]
  B[0],B[-1]=one,last
  return B
known_mat([2,43,14,56,6,5,6],200)

[-13557, 14214, -18344, 9806, 1605]

In [ ]:
# diagonal determinants
import numpy as np # this fxn determines the diagonal elements in a tridiagonal matrix
def diag(matrix):
  a=np.array(matrix)
  r,c=a.shape[0],a.shape[1]
  aa,bb,cc=[],[],[]
  for i in range(r-1):
    aa.append(a[i+1][i])
    cc.append(a[i][i+1])
  for i in range(r):
    bb.append(a[i][i])
  return (np.array(aa),np.array(bb),np.array(cc))
diag([[10,2,0,0],[3,10,4,0],[0,1,7,5],[0,0,3,4]])

(array([3, 1, 3]), array([10, 10,  7,  4]), array([2, 4, 5]))

In [ ]:
# Thomas algorithm solver
#import numpy as np

## Tri Diagonal Matrix Algorithm(a.k.a Thomas algorithm) solver
import numpy as np
from numba import jit    

@jit
def TDMAsolver(a,b,c,d):
    n = len(d)
    w= np.zeros(n-1,float)
    g= np.zeros(n, float)
    p = np.zeros(n,float)
    
    w[0] = c[0]/b[0]
    g[0] = d[0]/b[0]

    for i in range(1,n-1):
        w[i] = c[i]/(b[i] - a[i-1]*w[i-1])
    for i in range(1,n):
        g[i] = (d[i] - a[i-1]*g[i-1])/(b[i] - a[i-1]*w[i-1])
    p[n-1] = g[n-1]
    for i in range(n-1,0,-1):
        p[i-1] = g[i-1] - w[i-1]*p[i]
    return p

In [ ]:
import pandas
def cn_solver(pre_time,dt,dx,vis,compress,por,k):    # the cn solver that creates the solution for us along with plots
  import numpy as np
  import pandas as pd
  import matplotlib.pyplot as plt
  A=(k*dt/(vis*por*compress*(dx**2)))*0.5
  m=matrix_cr(len(pre_time),A)
  known=known_mat(pre_time,A)
  diagn=diag(m)
  aa,bb,cc,dd=diagn[0],diagn[1],diagn[2],np.array(known)
  solu=TDMAsolver(aa,bb,cc,dd)
  return solu
a=cn_solver([3000,4000,4000,4000,4000],0.01,0.1,1,1,1,1)
a.tolist()

[3464.2857142857147, 3857.142857142858, 3964.2857142857147]

In [ ]:
#  cn solver using time step
def cn_time(time_limit,pre_time,dt,dx,vis,compress,por,k):
  t,arr,P_data,P_plot=0,pre_time,[],[] # setting arr as initial values of pressure
  while t<=time_limit:
     solu=cn_solver(arr,dt,dx,vis,compress,por,k)
     P_data.append([pre_time[0]]+solu.tolist()+[pre_time[-1]])
     P_plot.append(solu.tolist())
     arr=[pre_time[0]]+solu.tolist()+[pre_time[-1]]
     t+=dt
  return (P_data,P_plot)
a=cn_time(0.05,[3000,4000,4000,4000,4000],0.01,0.1,1,1,1,1)
print(a[1])
# we do for dt= 0.1,0.5,1.0,2.5 and 5 


#'''import matplotlib.pyplot as plt
#from statistics import mean
#t=[i for i in range(1,2)]
#for i in range(len(a[1])):
#  a[1][i]=mean(a[1][i])
#plt.plot(t,a[1])'''

[[3464.2857142857147, 3857.142857142858, 3964.2857142857147], [3382.65306122449, 3673.4693877551026, 3882.65306122449], [3318.513119533528, 3600.5830903790093, 3818.513119533528], [3288.525614327364, 3553.5193669304463, 3788.5256143273637], [3270.794906884037, 3529.6602606057, 3770.7949068840367], [3261.4450611564916, 3516.119984020264, 3761.4450611564907]]


In [ ]:
#class CN(por,perm,vis,compress)

In [ ]:
# explicit
def explicit(pre_time,dt,dx,vis,compress,por,k):
  n=len(pre_time)
  arr=pre_time
  ans,A=[],(k*dt/(vis*por*compress*(dx**2)))
  for i in range(n-2):
    a=arr[i+1]+(A*(arr[i]+arr[i+2]-2*arr[i+1]))
    ans.append(a)
  return ans
explicit([100,200,222,333,300],1,1,1.44,0.22,0.44,200)
def exp_time(time_limit,pre_time,dt,dx,vis,compress,por,k):
  t,P_data,P_plot,arr=0,[],[],pre_time
  while t<=time_limit:
    solu=explicit(arr,dt,dx,vis,compress,por,k)
    P_data.append([pre_time[0]]+solu+[pre_time[-1]])
    P_plot.append(solu)
    arr=[pre_time[0]]+solu+[pre_time[-1]]
    t+=dt
  return (P_data,P_plot)
#exp_time(0.5,[3000,4000,4000,4000,4000],0.01,0.1,1,1,1,1)[1]

In [ ]:
# implicit cell
def mat_imp(n,A):
  tri_co,c,mat=(1+2*A),n-2,[]
  d=c-3# number of zeros after real coefficients in matrix from second row
  one=[tri_co,-A]+[0 for i in range(c-2)]
  last=[0 for i in range(c-2)]+[-A,tri_co]
  for i in range(c-2):
    a=[0]*(i)+[-A,tri_co,-A]+[0]*(d-i)
    mat.append(a)
  return [one]+mat+[last]
def mat_know_imp(arr,A):
  ans=arr[1:-1]
  ans[0]=ans[0]+arr[0]*A
  ans[-1]=ans[-1]+arr[-1]*A
  return ans
#mat_imp(5,100)
def implicit(pre_time,dt,dx,vis,compress,por,k):
  A=(k*dt/(vis*por*compress*(dx**2)))# this is ratio of dx**2/dt
  mat=mat_imp(len(pre_time),A)
  know=mat_know_imp(pre_time,A)
  diagn=diag(mat)
  aa,bb,cc,dd=diagn[0],diagn[1],diagn[2],np.array(know)
  solu=TDMAsolver(aa,bb,cc,dd)
  return solu.tolist()
def imp_time(time_limit,pre_time,dt,dx,vis,compress,por,k):
  t,P_data,P_plot,arr=0,[],[],pre_time
  while t<=time_limit:
    solu=implicit(arr,dt,dx,vis,compress,por,k)
    P_data.append([pre_time[0]]+solu+[pre_time[-1]])
    P_plot.append(solu)
    arr=[pre_time[0]]+solu+[pre_time[-1]]
    t+=dt
  return (P_data,P_plot)
a=imp_time(0.5,[3000,4000,4000,4000,4000],1,0.1,1,1,1,1)
a[1]

[[3258.6208704528926, 3509.8279496103137, 3756.133308263838]]

In [ ]:
# data storage to csv files
import csv
import pandas as pd
header=[i for i in range(12)]

# explicit

# implicit

#crank nicolson
file_imp=cn_time(5,[3000,4000,4000,4000,4000,4000,4000,4000,4000,4000,4000],1.0,0.1,1,1,1,1)[0]# at dt=2.5
a=pd.DataFrame(file_imp)
a.to_excel('cn_1.0.xlsx')
#for i in range(1,len(file_cn)):
 # a.loc[i]=file_cn[i]